# Import Dependencies

In [14]:
import pandas as pd

# Read data file to dict of pandas dataframes

In [15]:
db_file = './data/question_database_schema.xlsx'
dict_of_dfs = pd.read_excel(db_file, sheet_name = None)

print(dict_of_dfs.keys())

dict_keys(['learning_objectives', 'course_objective_description', 'questions', 'answer_choices', 'exams', 'exams_learning_objectives', 'distractor_type', 'student_question_responses'])


In [16]:
print(dict_of_dfs['questions'])

    question_id learning_objective_id other_possible_choices certainty
0          1A01                    66                    NaN       NaN
1          1A02                    25                    NaN       NaN
2          1A03                     3                      4       NaN
3          1A04                     4                    NaN       NaN
4          1A05                     2                    NaN       NaN
..          ...                   ...                    ...       ...
204        4C22                    21                    NaN       NaN
205        4C23                    27                    NaN       NaN
206        4C24                    28                    NaN       NaN
207        4C25                    29                    NaN       NaN
208        4C26                    29                    NaN       NaN

[209 rows x 4 columns]


# Exam descriptions

### Number of exams: 4

- Exam 1: 
    - 2 forms (A, B)
    - 16 Questions
      - 2 with 5 options
      - 14 with 4 options
- Exam 2: 3 forms (A, B, C)
    - 16 Questions
      - 1 with 3 options
      - 15 with 4 options
- Exam 3: 3 forms (A, B, C)
    - 17 Questions
      - 17 with 4 options
- Exam 4: 3 forms (A, B, C)
    - 26 Questions
      - 25 with 4 options
      - 1 with 3 options (which form of the exam do you have?)


In [17]:
completed_answer_choices = dict_of_dfs['answer_choices']
student_responses        = dict_of_dfs['student_question_responses']
#convert option_id to number indexed at 1.
if 'A' in pd.unique(completed_answer_choices['option_id']):
    completed_answer_choices['option_id'] = completed_answer_choices['option_id'] = [ ord(letter) - 64 for letter in completed_answer_choices['option_id'] ]

student_responses = student_responses[student_responses['question_id'].isin(pd.unique(completed_answer_choices['question_id']))]

student_responses_with_details = pd.merge(
    left=student_responses, 
    right=completed_answer_choices,
    how='left',
    left_on=['question_id', 'selected_option'],
    right_on=['question_id', 'option_id'],
)

number_of_distractors_chosen = len(student_responses_with_details[student_responses_with_details['is_distractor'] > .5])
number_of_correct_answers_chosen = len(student_responses_with_details[student_responses_with_details['is_distractor'] < .5])
number_of_questions = len(student_responses_with_details)

print(f"Number of questions: {number_of_questions}.\nNumber of correct answers chosen: {number_of_correct_answers_chosen}.\nNumber of distractors chosen: {number_of_distractors_chosen}")

Number of questions: 16853.
Number of correct answers chosen: 12535.
Number of distractors chosen: 4247


In [18]:
number_of_solution_based_distractors = len(student_responses_with_details[student_responses_with_details['distractor_type'] == 2])

In [19]:
print(f"Number of distractors chosen: {number_of_distractors_chosen}. Number of solution-based distractors chosen: {number_of_solution_based_distractors}")

Number of distractors chosen: 4247. Number of solution-based distractors chosen: 1799


In [20]:
completed_answer_choices

,question_id,option_id,is_distractor,distractor_type,distractor_learning_objective_understanding,notes
0,1A01,1,1,2,66.0,Ignore middle term
1,1A01,2,1,2,66.0,Ignore middle term
2,1A01,3,1,5,66.0,7*8=56 vs 54
3,1A01,4,1,5,66.0,7*8=56 vs 54
4,1A01,5,0,0,66.0,Trinomial is prime
...,...,...,...,...,...,...
820,4C25,4,1,6,NaN,NaN
821,4C26,1,1,1,NaN,NaN
822,4C26,2,1,7,NaN,NaN
823,4C26,3,0,0,NaN,NaN


In [21]:
exam_question_distractor_count_frame = completed_answer_choices.groupby(by = 'distractor_type').count()["question_id"].reset_index().rename(columns = {"question_id": "count", "distractor_type": "distractor_id"})

exam_question_distractor_count_frame = pd.merge(
    left=exam_question_distractor_count_frame, 
    right=dict_of_dfs['distractor_type'],
    how='left',
    left_on=['distractor_id'],
    right_on=['distractor_id'],
)
exam_question_distractor_count_frame["percent"] = exam_question_distractor_count_frame["count"] / exam_question_distractor_count_frame["count"].sum() 

display(exam_question_distractor_count_frame)

,distractor_id,count,distractor_type,explanation,percent
0,0,206,solution,Solution based on expected student thinking fo...,0.249697
1,1,242,solution_based,Answer based on manipulating the solution that...,0.293333
2,2,211,question_learning_objective_understanding,Answer based on level of conception associated...,0.255758
3,3,1,other_learning_objective_understanding,Answer based on level of conception associated...,0.001212
4,4,3,representation,Answer based on a different representation tha...,0.003636
5,5,34,mechanics_based,Answer based on mechanical error with expected...,0.041212
6,6,40,unsure,It be like that sometimes,0.048485
7,7,85,distractor_based,Answer based on manipulating another option,0.103030
8,8,3,concept-based doppleganger,Answer based on conception but modified to loo...,0.003636


In [22]:
student_responses_distractor_selection_counts = student_responses_with_details.groupby(by = 'distractor_type').count()["question_id"].reset_index().rename(columns = {"question_id": "count", "distractor_type": "distractor_id"})
student_responses_distractor_selection_counts = pd.merge(
    left=student_responses_distractor_selection_counts, 
    right=dict_of_dfs['distractor_type'],
    how='left',
    left_on=['distractor_id'],
    right_on=['distractor_id'],
)
student_responses_distractor_selection_counts["percent"] = student_responses_distractor_selection_counts["count"] / student_responses_distractor_selection_counts["count"].sum() 
display(student_responses_distractor_selection_counts)

,distractor_id,count,distractor_type,explanation,percent
0,0.0,12535,solution,Solution based on expected student thinking fo...,0.746931
1,1.0,1613,solution_based,Answer based on manipulating the solution that...,0.096115
2,2.0,1799,question_learning_objective_understanding,Answer based on level of conception associated...,0.107198
3,3.0,9,other_learning_objective_understanding,Answer based on level of conception associated...,0.000536
4,4.0,32,representation,Answer based on a different representation tha...,0.001907
5,5.0,264,mechanics_based,Answer based on mechanical error with expected...,0.015731
6,6.0,207,unsure,It be like that sometimes,0.012335
7,7.0,305,distractor_based,Answer based on manipulating another option,0.018174
8,8.0,18,concept-based doppleganger,Answer based on conception but modified to loo...,0.001073


# Effective Distractor Analysis

In [23]:
distractor_selection_counts = student_responses_with_details.groupby(by = ['question_id', 'selected_option']).count()
distractor_selection_counts = distractor_selection_counts.rename(columns = {"student_id": "count"})["count"].to_frame().reset_index()
distractor_selection_counts = pd.merge(
    left=completed_answer_choices[["question_id", "option_id", "is_distractor"]],
    right = distractor_selection_counts,
    how='left',
    left_on=['question_id', 'option_id'],
    right_on=['question_id', 'selected_option'],
)
distractor_selection_counts = distractor_selection_counts.drop(columns = ["selected_option"])
distractor_selection_counts = distractor_selection_counts.fillna(0)
distractor_selection_counts["percent"] = distractor_selection_counts["count"] 
distractor_selection_counts["exam_id"] = distractor_selection_counts["question_id"].str[:2]
question_id_columns = pd.unique(distractor_selection_counts["question_id"].values)

for question_id in question_id_columns:
    question_selection_frame = distractor_selection_counts[distractor_selection_counts["question_id"].isin([question_id])]
    question_selection_frame.loc[:, "percent"] = question_selection_frame["count"]/question_selection_frame["count"].sum()
    distractor_selection_counts[distractor_selection_counts["question_id"].isin([question_id])] = question_selection_frame


## Percentage of Distractors Chosen by Form

In [24]:
exam_ids = pd.unique(distractor_selection_counts["exam_id"].values)
distractors_chosen_list = []
for exam_id in exam_ids:
    exam_distractors_chosen_dict = dict()
    exam_distractors_chosen_dict["exam_id"] = exam_id
    exam_selection_frame = distractor_selection_counts[distractor_selection_counts["exam_id"].isin([exam_id])]
    exam_selection_frame = exam_selection_frame[exam_selection_frame["is_distractor"] == 1]
    exam_distractors_chosen_dict["never_chosen"] = len(exam_selection_frame[exam_selection_frame["percent"] == 0])
    exam_distractors_chosen_dict["rarely_chosen"] = len(exam_selection_frame[(exam_selection_frame["percent"] > 0) & (exam_selection_frame["percent"] <= 0.05)])
    exam_distractors_chosen_dict["sometimes_chosen"] = len(exam_selection_frame[exam_selection_frame["percent"] > 0.05])
    exam_distractors_chosen_dict["total_distractors"] = len(exam_selection_frame)
    distractors_chosen_list.append(exam_distractors_chosen_dict)
exam_distractors_chosen_frame = pd.DataFrame(distractors_chosen_list)
exam_distractors_chosen_frame["never_chosen_percent"] = exam_distractors_chosen_frame["never_chosen"] / exam_distractors_chosen_frame["total_distractors"]
exam_distractors_chosen_frame["rarely_chosen_percent"] = exam_distractors_chosen_frame["rarely_chosen"] / exam_distractors_chosen_frame["total_distractors"]
exam_distractors_chosen_frame["sometimes_chosen_percent"] = exam_distractors_chosen_frame["sometimes_chosen"] / exam_distractors_chosen_frame["total_distractors"]

print(exam_distractors_chosen_frame)

exam_distractors_chosen_frame["exam_form_id"] = exam_distractors_chosen_frame["exam_id"]
exam_distractors_chosen_frame["form"] = exam_distractors_chosen_frame["exam_id"].str[1]
exam_distractors_chosen_frame["exam_id"] = exam_distractors_chosen_frame["exam_id"].str[0]


   exam_id  never_chosen  rarely_chosen  sometimes_chosen  total_distractors  \
0       1A             1             18                31                 50   
1       1B             4             18                28                 50   
2       2A             3             18                26                 47   
3       2B             3             17                27                 47   
4       2C             0             18                29                 47   
5       3A             7             20                24                 51   
6       3B             4             27                20                 51   
7       3C            11             21                19                 51   
8       4A             7             25                43                 75   
9       4B            13             22                40                 75   
10      4C            17             30                28                 75   

    never_chosen_percent  rarely_chosen

## Number of effective distractors per question per form

In [ ]:
def create_effective_distractor_counts_list():
    return [0, 0, 0, 0, 0]

distractor_selection_counts["is_effective"] = distractor_selection_counts["percent"] > 0.05

effective_distractor_count_dict = dict()

for question_id in question_id_columns:
    question_selection_frame = distractor_selection_counts[distractor_selection_counts["question_id"].isin([question_id])]
    question_selection_frame = question_selection_frame[question_selection_frame["is_distractor"] == 1]
    exam_id = question_selection_frame["exam_id"].values[0]
    number_of_effective_distractors = question_selection_frame["is_effective"].sum()
    if exam_id not in effective_distractor_count_dict.keys():
        effective_distractor_count_dict[exam_id] = create_effective_distractor_counts_list()
    effective_distractor_count_dict[exam_id][number_of_effective_distractors] += 1

for key in effective_distractor_count_dict.keys():
    list_sum = sum(effective_distractor_count_dict[key])
    for i in range(len(effective_distractor_count_dict[key])):
        effective_distractor_count_dict[key][i] = effective_distractor_count_dict[key][i] / list_sum
    print(key, effective_distractor_count_dict[key])

    question_id  option_id  is_distractor  count   percent exam_id  \
0          1A01          1              1    1.0  0.008547      1A   
1          1A01          2              1    3.0  0.025641      1A   
2          1A01          3              1   29.0  0.247863      1A   
3          1A01          4              1    2.0  0.017094      1A   
4          1A01          5              0   82.0  0.700855      1A   
..          ...        ...            ...    ...       ...     ...   
820        4C25          4              1    7.0  0.094595      4C   
821        4C26          1              1    0.0  0.000000      4C   
822        4C26          2              1    2.0  0.027027      4C   
823        4C26          3              0   69.0  0.932432      4C   
824        4C26          4              1    3.0  0.040541      4C   

     is_effective  
0           False  
1           False  
2            True  
3           False  
4            True  
..            ...  
820          True  

# Item Difficulty